<a href="https://colab.research.google.com/github/GarlGuo/ChocoSGD/blob/master/Testing_of_Conditional_Text_Generation_with_GPT_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Related article: https://www.ivanlai.project-ds.net/post/conditional-text-generation-by-fine-tuning-gpt-2

Preprocessing code in [this](https://github.com/ivanlai/Conditional_Text_Generation) Github repository.

### Install and import libraries

In [ ]:
%%time
%%capture
!pip install transformers

CPU times: user 83.9 ms, sys: 25 ms, total: 109 ms
Wall time: 9.97 s


Check GPU memory available (Colab could offer 12GB or 16GB). 

Our configuration works on 16GB. The batch size needs to be reduced if only 12GB were available.




In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
import os
import io
import requests
import numpy as np
import pandas as pd
import re
import zipfile
import random
import time
import csv
import datetime
from itertools import compress
from collections import Counter, defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from transformers import AutoTokenizer, AutoConfig, AutoModelForPreTraining, \
                         AdamW, get_linear_schedule_with_warmup, \
                         TrainingArguments, BeamScorer, Trainer

import torch
from torch.utils.data import Dataset, random_split, DataLoader, \
                             RandomSampler, SequentialSampler

from IPython.display import clear_output

print(f"PyTorch version: {torch.__version__}")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

PyTorch version: 1.12.1+cu113


### Configurations

In [ ]:
DEBUG           = False

INPUT_DIR       = 'articles'

USE_APEX        = True
APEX_OPT_LEVEL  = 'O1'

MODEL           = 'gpt2' #{gpt2, gpt2-medium, gpt2-large, gpt2-xl}

UNFREEZE_LAST_N = 6 #The last N layers to unfreeze for training

SPECIAL_TOKENS  = { "bos_token": "<|BOS|>",
                    "eos_token": "<|EOS|>",
                    "unk_token": "<|UNK|>",                    
                    "pad_token": "<|PAD|>",
                    "sep_token": "<|SEP|>",
                    "additional_special_tokens":['[C1]','[C2]']}
                    
MAXLEN          = 768  #{768, 1024, 1280, 1600}

TRAIN_SIZE      = 0.8

if USE_APEX:
    TRAIN_BATCHSIZE = 4
    BATCH_UPDATE    = 16
else:
    TRAIN_BATCHSIZE = 2
    BATCH_UPDATE    = 32

EPOCHS          = 2
LR              = 5e-4
EPS             = 1e-8
WARMUP_STEPS    = 1e2

SEED            = 2020

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

### Download News Aggregator Dataset

https://archive.ics.uci.edu/ml/datasets/News+Aggregator

In [ ]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip'
r = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

In [ ]:
columns = ['ID',
           'TITLE',
           'URL',
           'PUBLISHER',
           'CATEGORY', #News category (b = business, t = science and technology, e = entertainment, m = health)
           'Alphanumeric ID',
           'HOSTNAME Url',
           'TIMESTAMP']

df = pd.read_csv("newsCorpora.csv", sep='\t', header=None, names=columns)
print(f"df size: {len(df) :,}")

df.head(2)

df size: 422,419


,ID,TITLE,URL,PUBLISHER,CATEGORY,Alphanumeric ID,HOSTNAME Url,TIMESTAMP
0,1,"Fed official says weak data caused by weather,...",http://www.latimes.com/business/money/la-fi-mo...,Los Angeles Times,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.latimes.com,1394470370698
1,2,Fed's Charles Plosser sees high bar for change...,http://www.livemint.com/Politics/H2EvwJSK2VE6O...,Livemint,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.livemint.com,1394470371207


### Download articles

In [ ]:
#Download news articles
!gdown --id 1FqqDZOVd8_LfOEA_2DEY5W70-DpQVF-F
!unzip -q '/content/news_articles.zip'

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1FqqDZOVd8_LfOEA_2DEY5W70-DpQVF-F
To: /content/news_articles.zip
100% 49.2M/49.2M [00:00<00:00, 142MB/s]


In [ ]:
%%time

data = dict()            
for root, dirs, files in os.walk(INPUT_DIR, topdown=True):
    t0 = time.time()

    for i, f in enumerate(files):
        #id, category, title, keywords, text
        id = int(f[:-4])        
        tmp = df[['CATEGORY', 'TITLE']][df.ID==id].values
        category, title = tmp[0][0], tmp[0][1]

        with open(f'{INPUT_DIR}/{f}', "r") as infile:
            text = infile.read()
        
        data[id] = [title, text]

        if i%1000==0 and i>0:
            clear_output(wait=True)
            print(f"({os.getpid()}) Items processed: {i :,}/{len(files):,}; {(time.time()-t0)/60 :.1f} minutes")

            if DEBUG:
                break

print(f"Number of articles: {len(data) :,}")

(140) Items processed: 1,000/39,024; 0.3 minutes
(140) Items processed: 2,000/39,024; 0.6 minutes
(140) Items processed: 3,000/39,024; 0.8 minutes
(140) Items processed: 4,000/39,024; 1.1 minutes
(140) Items processed: 5,000/39,024; 1.4 minutes
(140) Items processed: 6,000/39,024; 1.7 minutes
(140) Items processed: 7,000/39,024; 1.9 minutes
(140) Items processed: 8,000/39,024; 2.2 minutes
(140) Items processed: 9,000/39,024; 2.5 minutes
(140) Items processed: 10,000/39,024; 2.7 minutes
(140) Items processed: 11,000/39,024; 3.0 minutes
(140) Items processed: 12,000/39,024; 3.3 minutes
(140) Items processed: 13,000/39,024; 3.6 minutes
(140) Items processed: 14,000/39,024; 3.8 minutes
(140) Items processed: 15,000/39,024; 4.1 minutes
(140) Items processed: 16,000/39,024; 4.4 minutes
(140) Items processed: 17,000/39,024; 4.7 minutes
(140) Items processed: 18,000/39,024; 4.9 minutes
(140) Items processed: 19,000/39,024; 5.2 minutes
(140) Items processed: 20,000/39,024; 5.5 minutes
(140) Ite

### Download Keywords 
Keywords of these articles have been extracted offline

In [ ]:
#Download Keywords
!gdown --id 1C1WWvnt2egzhRmVXMSJhARz5GOLDGzMC
!unzip -q '/content/keywords.csv.zip'

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1C1WWvnt2egzhRmVXMSJhARz5GOLDGzMC
To: /content/keywords.csv.zip
100% 746k/746k [00:00<00:00, 37.0MB/s]


In [ ]:
def read_keywords():
    keywords = dict()
    with open('keywords.csv', newline='') as f:
        reader = csv.reader(f, delimiter=',')
        for row in reader:        
            keywords[int(row[0])] = row[1:]          
    print(f"Number of entries in keywords: {len(keywords) :,}")  
    return keywords

In [ ]:
%%time
keywords = read_keywords()

all_keywords = set()
for k, v in keywords.items():
    for w in v:
        all_keywords.add(w)
        
for id in data.keys():
    data[id].append(keywords[id])

print(f"Number of unique keywords: {len(all_keywords) :,}")  

Number of entries in keywords: 39,024
Number of unique keywords: 29,291
CPU times: user 164 ms, sys: 19.1 ms, total: 183 ms
Wall time: 177 ms


### Datasets and loaders

In [ ]:
class myDataset(Dataset):

    def __init__(self, data, tokenizer, randomize=True):

        title, text, keywords = [], [], []
        for k, v in data.items():
            title.append(v[0])
            text.append(v[1])
            keywords.append(v[2])

        self.randomize = randomize
        self.tokenizer = tokenizer 
        self.title     = title
        self.text      = text
        self.keywords  = keywords  

    #---------------------------------------------#

    @staticmethod
    def join_keywords(keywords, randomize=True):
        N = len(keywords)

        #random sampling and shuffle
        if randomize: 
            M = random.choice(range(N+1))
            keywords = keywords[:M]
            random.shuffle(keywords)

        return ','.join(keywords)

    #---------------------------------------------#

    def __len__(self):
        return len(self.text)

    #---------------------------------------------#
    
    def __getitem__(self, i):
        keywords = self.keywords[i].copy()
        kw = self.join_keywords(keywords, self.randomize)
        
        input = SPECIAL_TOKENS['bos_token'] + SPECIAL_TOKENS['additional_special_tokens'][0] + SPECIAL_TOKENS['additional_special_tokens'][1] + \
                self.title[i] + \
                SPECIAL_TOKENS['sep_token'] + kw + SPECIAL_TOKENS['sep_token'] + \
                self.text[i] + SPECIAL_TOKENS['eos_token']

        encodings_dict = tokenizer(input,                                   
                                   truncation=True, 
                                   max_length=MAXLEN, 
                                   padding="max_length")   
        
        input_ids = encodings_dict['input_ids']
        attention_mask = encodings_dict['attention_mask']
        
        return {'label': torch.tensor(input_ids),
                'input_ids': torch.tensor(input_ids), 
                'attention_mask': torch.tensor(attention_mask)}

In [ ]:
def split_data(data, S=TRAIN_SIZE):
    # Shuffle ids
    ids = list(data.keys())
    random.shuffle(ids)

    # Split into training and validation sets    
    train_size = int(S * len(data))

    train_ids = ids[:train_size]
    val_ids = ids[train_size:]

    train_data = dict()
    for id in train_ids:
        train_data[id] = data[id]

    val_data = dict()
    for id in val_ids:
        val_data[id] = data[id]

    return train_data, val_data

### Loading Tokenizer, Config and Model

In [ ]:
def get_tokenier(special_tokens=None):
    tokenizer = AutoTokenizer.from_pretrained(MODEL) #GPT2Tokenizer

    if special_tokens:
        tokenizer.add_special_tokens(special_tokens)
        print("Special tokens added")
    return tokenizer

def get_model(tokenizer, special_tokens=None, load_model_path=None):

    #GPT2LMHeadModel
    if special_tokens:
        config = AutoConfig.from_pretrained(MODEL, 
                                            bos_token_id=tokenizer.bos_token_id,
                                            eos_token_id=tokenizer.eos_token_id,
                                            sep_token_id=tokenizer.sep_token_id,
                                            pad_token_id=tokenizer.pad_token_id,
                                            output_hidden_states=False)
    else: 
        config = AutoConfig.from_pretrained(MODEL,                                     
                                            pad_token_id=tokenizer.eos_token_id,
                                            output_hidden_states=False)    

    #----------------------------------------------------------------#
    model = AutoModelForPreTraining.from_pretrained(MODEL, config=config)

    if special_tokens:
        #Special tokens added, model needs to be resized accordingly
        model.resize_token_embeddings(len(tokenizer))

    if load_model_path:
        model.load_state_dict(torch.load(load_model_path))

    model.cuda()
    return model

In [ ]:
%%time

tokenizer = get_tokenier(special_tokens=SPECIAL_TOKENS)
model = get_model(tokenizer, 
                  special_tokens=SPECIAL_TOKENS,
                #   load_model_path='pytorch_model.bin'
                 )

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Special tokens added


Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

RuntimeError: ignored

In [ ]:
# - Freeze selective layers:
# - Freeze all layers except last n:
for parameter in model.parameters():
    parameter.requires_grad = False

for i, m in enumerate(model.transformer.h):        
    #Only un-freeze the last n transformer blocks
    if i+1 > 12 - UNFREEZE_LAST_N:
        for parameter in m.parameters():
            parameter.requires_grad = True 

for parameter in model.transformer.ln_f.parameters():        
    parameter.requires_grad = True

for parameter in model.lm_head.parameters():        
    parameter.requires_grad = True

NameError: ignored

In [ ]:
train_data, val_data = split_data(data)

train_dataset = myDataset(train_data, tokenizer)
val_dataset = myDataset(val_data, tokenizer, randomize=False)

f'There are {len(train_dataset) :,} samples for training, and {len(val_dataset) :,} samples for validation testing'

### Fine-tune GPT2 using Trainer

In [ ]:
%%time

training_args = TrainingArguments(
    output_dir="/content/",
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=TRAIN_BATCHSIZE,
    per_device_eval_batch_size=TRAIN_BATCHSIZE,
    gradient_accumulation_steps=BATCH_UPDATE,
    evaluation_strategy="epoch",
    fp16=True,
    fp16_opt_level=APEX_OPT_LEVEL,
    warmup_steps=WARMUP_STEPS,    
    learning_rate=LR,
    adam_epsilon=EPS,
    weight_decay=0.01,        
    save_total_limit=1,
    load_best_model_at_end=False,     
)

#---------------------------------------------------#
trainer = Trainer(
    model=model,
    args=training_args,    
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

#---------------------------------------------------#
trainer.train()
trainer.save_model()    

In [ ]:
# Save to G-Drive ----------------------------------#
# !cp -r 'pytorch_model.bin' '/content/drive/MyDrive/Colab Notebooks/Text Generation/pytorch_model_V2.bin'

### Generating text with Fine-tuned GPT-2 model

In [ ]:
# !cp -r '/content/drive/MyDrive/Colab Notebooks/Text Generation/pytorch_model_V2.bin' 'pytorch_model.bin' 

In [ ]:
tokenizer = get_tokenier(special_tokens=SPECIAL_TOKENS)
model = get_model(tokenizer, 
                  special_tokens=SPECIAL_TOKENS,
                  load_model_path='pytorch_model.bin')

In [ ]:
title = "We got a lot of grief when our photo became a meme"
keywords = ['train', 'lads', 'drinking', 'picture', 'funny', 'instagram']
kw = myDataset.join_keywords(keywords, randomize=False)

prompt = SPECIAL_TOKENS['bos_token']
        #   + title + \
        #  SPECIAL_TOKENS['sep_token'] + kw + SPECIAL_TOKENS['sep_token']
         
generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
device = torch.device("cuda")
generated = generated.to(device)

model.eval();

In [ ]:
# Top-p (nucleus) text generation (10 samples):
sample_outputs = model.generate(generated, 
                                do_sample=True,   
                                min_length=50, 
                                max_length=MAXLEN,
                                top_k=30,                                 
                                top_p=0.7,        
                                temperature=0.9,
                                repetition_penalty=2.0,
                                num_return_sequences=10
                                )

for i, sample_output in enumerate(sample_outputs):
    text = tokenizer.decode(sample_output, skip_special_tokens=True)
    a = len(title) + len(','.join(keywords))    
    print("{}: {}\n\n".format(i+1,  text[a:]))

In [ ]:
# gen_sequences = sample_outputs.sequences[:, input_ids.shape[-1]:]
generated.shape[-1]

In [ ]:
# Beam-search text generation:
sample_outputs = model.generate(generated, 
                                return_dict_in_generate=True,
                                output_scores=True,
                                # do_sample=True,   
                                max_length=MAXLEN,                                                      
                                num_beams=5,
                                repetition_penalty=5.0,
                                early_stopping=True,      
                                num_return_sequences=5
                                )

# for i, sample_output in enumerate(sample_outputs):
#     text = tokenizer.decode(sample_output, skip_special_tokens=True)
#     a = len(title) + len(','.join(keywords))    
#     print("{}: {}\n\n".format(i+1,  text[a:]))

# gen_sequences = sample_outputs.sequences[:, input_ids.shape[-1]:]

In [ ]:
gen_sequences = sample_outputs.sequences[:, generated.shape[-1]:]
gen_sequences

In [ ]:
for i, gen_seq in enumerate(gen_sequences):
  print(f"{i}th gen_seq is:")
  print(tokenizer.decode(gen_seq))

In [ ]:
probs = torch.stack(sample_outputs.scores, dim=1).softmax(-1)
# print(sample_outputs.scores)
probs

In [ ]:
tokenizer.encode(SPECIAL_TOKENS["additional_special_tokens"])

### Generating text with raw GPT2

In [ ]:
tokenizer = get_tokenier()
model = get_model(tokenizer)

In [ ]:
prompt = title

generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
device = torch.device("cuda")
generated = generated.to(device)

model.eval()
sample_outputs = model.generate(generated, 
                                do_sample=True,   
                                max_length=MAXLEN,                                                      
                                num_beams=5,
                                repetition_penalty=5.0,
                                early_stopping=True,      
                                num_return_sequences=1
                                )

for i, sample_output in enumerate(sample_outputs):
    print("{}: {}\n\n".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))